In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from src.feature_engineering import (
    apply_feature_engineering,
    drop_all_zero_entries,
    choose_acc_ids,
)
import yaml

acc_config_path = Path("../config/acc_config.yaml")
with open(acc_config_path, "r") as yaml_file:
    acc_config = yaml.safe_load(yaml_file)

In [2]:
file_path = Path("../data/final/merged_complete.csv")
df = pd.read_csv(file_path, index_col=None, header=0)

df = df.sort_values(by="Year")


In [3]:
df.head()

In [4]:
import matplotlib.pyplot as plt
import os
import io
from PIL import Image
plt.style.use('default')
output_dir = 'time_series_additional_plots'
os.makedirs(output_dir, exist_ok=True)

buffer = io.StringIO()
df.info(buf=buffer)
info_str = buffer.getvalue()

fig, ax = plt.subplots(figsize=(10, 8))
ax.text(0, 1, info_str, fontsize=12, fontfamily='monospace', va='top')
ax.axis('off')

png_output_path = f"{output_dir}/dataframe_info.png"
plt.savefig(png_output_path, bbox_inches='tight')
plt.close()

In [5]:

df = apply_feature_engineering(df)
df = drop_all_zero_entries(df)
CATEGORY = "Aufwand"
df = choose_acc_ids(df, acc_config.get(CATEGORY)) 

df['Year'] = df['Year'].astype(int)
df['Realized'] = pd.to_numeric(df['Realized'], errors='coerce') / 1000000
df['Budget y'] = pd.to_numeric(df['Budget y'], errors='coerce') / 1000000

aggregated_data = df.groupby(['Year', 'Region'])[['Realized', 'Budget y']].sum().reset_index()

aggregated_data['Percentage Difference'] = ((aggregated_data['Budget y'] - aggregated_data['Realized']) / aggregated_data['Realized']).replace([np.inf, -np.inf], np.nan) * 100

In [6]:
from statsmodels.tsa.stattools import adfuller, bds

aggregated_dataZurich = aggregated_data[aggregated_data['Region'] == 'ZH']

result_adf = adfuller(aggregated_dataZurich["Realized"].values)
print(f'ADF Statistic: {result_adf[0]}')
print(f'p-value: {result_adf[1]}')
print('Critical Values:')
for key, value in result_adf[4].items():
    print(f'   {key}: {value}')

bds_result = bds(aggregated_dataZurich["Realized"])

bds_statistic, bds_pvalue = bds_result

print(f'BDS Statistic: {bds_statistic}')
print(f'p-value: {bds_pvalue}')

In [7]:
import seaborn as sns
import matplotlib.pyplot as plt
import os
from pathlib import Path

output_dir = 'time_series_additional_plots'
os.makedirs(output_dir, exist_ok=True)

plt.style.use('default')
sns.set(style="whitegrid") 

regions = aggregated_data['Region'].unique()

for region in regions:
    region_data = aggregated_data[aggregated_data['Region'] == region]
    
    plt.figure(figsize=(10, 4))
    sns.lineplot(data=region_data, x='Year', y='Percentage Difference', marker='o', linestyle='-', label=f'Percentage Difference - {region}')
    plt.title(f'Percentage Difference (Budget y to Realized) - {region} - Category {CATEGORY}', fontsize=16, color='black')
    plt.xlabel('Year', fontsize=12, color='black')
    plt.ylabel('Percentage Difference', fontsize=12, color='black')
    plt.axhline(0, color='grey', lw=0.8, ls='--') 
    plt.legend(title='Region', fontsize=10, title_fontsize='11', loc='best')
    plt.yticks(fontsize=10, color='black')
    years = region_data['Year'].unique()
    plt.xticks(ticks=years, labels=[int(year) for year in years], fontsize=10, color='black')
    
    plt.tight_layout()
    
    plt.savefig(f"{output_dir}/percentage_difference_{region}_{CATEGORY}.png", bbox_inches='tight')
    plt.close()


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os


output_dir = 'time_series_canton_budget_plot'
os.makedirs(output_dir, exist_ok=True)



def additional_stats(df):
    stats = pd.DataFrame()
    stats['mean'] = df.mean().round(2)
    stats['median'] = df.median().round(2)
    stats['min'] = df.min().round(2)
    stats['max'] = df.max().round(2)
    return stats

sns.set(style="whitegrid")
font_size = 12

cantons = df['Region'].unique()

for canton in cantons:
    canton_data = aggregated_data[aggregated_data['Region'] == canton]

    stats = additional_stats(canton_data[['Realized', 'Budget y', 'Percentage Difference']])
    stats_transposed = stats.transpose().round(2)
    stats_values = stats_transposed.values.tolist()
    stats_columns = stats_transposed.columns.tolist()
    stats_index = stats_transposed.index.tolist()

    updated_columns = [col + " (in millions)" if col != "Percentage Difference" else col for col in stats_columns]
    
    fig, (ax_header, ax1, ax2) = plt.subplots(3, 1, figsize=(14, 14), sharex=True,
                                              gridspec_kw={'height_ratios': [0.4, 0.6, 0.6]})

    ax_header.axis('off')
    ax_header.axis('tight')
    sns.lineplot(data=canton_data, x='Year', y='Realized', label='Realized', ax=ax1, color='blue')
    sns.lineplot(data=canton_data, x='Year', y='Budget y', label='Budget y', ax=ax1, color='green')
    ax1.fill_between(canton_data['Year'], canton_data['Realized'], canton_data['Budget y'], color='gray', alpha=0.3)

    for region in cantons:
        if region != canton:
            region_data = aggregated_data[aggregated_data['Region'] == region]
            sns.lineplot(data=region_data, x='Year', y='Percentage Difference', ax=ax2, color='gray', alpha=0.3)

    sns.lineplot(data=canton_data, x='Year', y='Percentage Difference', label='Percentage Difference from Realized to Budget', ax=ax2, color='purple')
    
    years = canton_data['Year'].unique()
    ax1.set_xticks(years)
    ax2.set_xticks(years)
    ax1.set_ylabel("Amount in millions", labelpad=10)
    ax1.get_yaxis().get_major_formatter().set_scientific(False)
    ax2.set_ylabel("Percentage Difference", labelpad=10)
    
    ax1.set_title(f"Realized vs. Budget in {canton}", fontsize=16)
    ax2.set_title(f"Percentage Difference (Budget y to Realized) - {canton}", fontsize=16)

    fig.text(0.066, 0.9, "Canton ", fontsize=30, color='dimgray')
    fig.text(0.175, 0.9, canton, fontsize=30, color='blue', weight='bold')
    fig.text(0.066, 0.88, f"Total years of data: {len(canton_data['Year'].unique())}", fontsize=font_size)
    fig.text(0.066, 0.865, f"Account position category: {CATEGORY}", fontsize=font_size)
    
    table_data = stats.transpose().values
    table = ax_header.table(cellText=stats_values,
                            colLabels=updated_columns,
                            rowLabels=stats_index,
                            cellLoc='center',
                            loc='upper right',
                            bbox=[0.5, 0.3, 0.5, 0.7])

    table.auto_set_font_size(False)
    table.set_fontsize(font_size)

    light_grey = '#f0f0f0'
    for (i, j), cell in table.get_celld().items():
        if j == -1: 
            cell.set_text_props(color='dimgrey')
        elif i == 0:  
            cell.set_text_props(color='dimgrey')
        else:  
            cell.set_text_props(color='black', size=10, ha='center', va='center_baseline')
            cell.set_facecolor(light_grey)
        cell.set_edgecolor('white')
        
    plt.style.use('default')
    plt.tight_layout()
    plt.savefig(f"{output_dir}/{canton}_budget_plot.png")
    plt.close()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os


output_dir = 'time_series_canton_budget_plot_comparison'
os.makedirs(output_dir, exist_ok=True)


sns.set(style="whitegrid")
font_size = 12

cantons = df['Region'].unique()

for canton in cantons:
    canton_data = aggregated_data[aggregated_data['Region'] == canton]

    sns.lineplot(data=canton_data, x='Year', y='Realized', label='Realized', ax=ax1, color='blue')
    sns.lineplot(data=canton_data, x='Year', y='Budget y', label='Budget y', ax=ax1, color='green')
    ax1.fill_between(canton_data['Year'], canton_data['Realized'], canton_data['Budget y'], color='gray', alpha=0.3)

    for region in cantons:
        if region != canton:
            region_data = aggregated_data[aggregated_data['Region'] == region]
            sns.lineplot(data=region_data, x='Year', y='Percentage Difference', ax=ax2, color='gray', alpha=0.3)

    sns.lineplot(data=canton_data, x='Year', y='Percentage Difference', label='Percentage Difference from Realized to Budget', ax=ax2, color='purple')
    
    years = canton_data['Year'].unique()
    ax1.set_xticks(years)
    ax2.set_xticks(years)
    ax1.set_ylabel("Amount in millions", labelpad=10)
    ax1.get_yaxis().get_major_formatter().set_scientific(False)
    ax2.set_ylabel("Percentage Difference", labelpad=10)
    
    ax1.set_title(f"Realized vs. Budget in {canton}", fontsize=16)
    ax2.set_title(f"Percentage Difference (Budget y to Realized) - {canton}", fontsize=16)

    
    table_data = stats.transpose().values
    table = ax_header.table(cellText=stats_values,
                            colLabels=updated_columns,
                            rowLabels=stats_index,
                            cellLoc='center',
                            loc='upper right',
                            bbox=[0.5, 0.3, 0.5, 0.7])

    table.auto_set_font_size(False)
    table.set_fontsize(font_size)

    light_grey = '#f0f0f0'
    for (i, j), cell in table.get_celld().items():
        if j == -1: 
            cell.set_text_props(color='dimgrey')
        elif i == 0:  
            cell.set_text_props(color='dimgrey')
        else:  
            cell.set_text_props(color='black', size=10, ha='center', va='center_baseline')
            cell.set_facecolor(light_grey)
        cell.set_edgecolor('white')
        
    plt.style.use('default')
    plt.tight_layout()
    plt.savefig(f"{output_dir}/{canton}_budget_plot.png")
    plt.close()

In [9]:
import seaborn as sns
import matplotlib.pyplot as plt

df_pivot = aggregated_data.pivot(index='Region', columns='Year', values='Percentage Difference')

plt.style.use('default')

plt.figure(figsize=(12, 10))
heatmap = sns.heatmap(df_pivot, annot=True, fmt=".1f", cmap='coolwarm', linewidths=.5, center=0, cbar_kws={'label': 'Percentage Difference'})

plt.title(f'Percent Deviation by Region and Year for Category {CATEGORY}', fontsize=18, color='black')
plt.xlabel('Year', fontsize=14, color='black')
plt.ylabel('Region', fontsize=14, color='black')

plt.xticks(rotation=45, fontsize=12, color='black')
plt.yticks(fontsize=12, color='black')

cbar = heatmap.collections[0].colorbar
cbar.ax.yaxis.label.set_size(14)
cbar.ax.yaxis.label.set_color('black')
cbar.ax.tick_params(labelsize=12, colors='black')

plt.savefig(f"{output_dir}/heatmap_PercentDeviation_{CATEGORY}.png")


In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path

output_dir = 'time_series_additional_plots'
os.makedirs(output_dir, exist_ok=True)
file_path = Path("../data/merged_canton/kantonsdaten_merged_filled.xlsx")
df = pd.read_excel(file_path, index_col=None, header=0)
df = df.rename(columns={'Jahr': 'Year', 'Kanton': 'Region'})
df = df.drop_duplicates(subset=['Year', 'Region'])
plt.style.use('default')

melted_data = df.melt(id_vars=['Year', 'Region'], var_name='Variable', value_name='Value')

variables = melted_data['Variable'].unique()

def plot_variable_chunk(chunk, chunk_index):
    chunk_data = melted_data[melted_data['Variable'].isin(chunk)]
    g = sns.FacetGrid(chunk_data, col="Variable", hue="Region", col_wrap=3, sharey=False, height=4, aspect=1.5)
    g.map(sns.lineplot, "Year", "Value")
    
    total_axes = len(g.axes.flat)
    n_rows = (total_axes + 2) // 3 

    for i, ax in enumerate(g.axes.flatten()):
        if i // 3 != n_rows - 1: 
            ax.tick_params(labelbottom=False)
        ax.set_xlabel('')
        ax.set_ylabel('Value', fontsize=10, color='black')
        ax.yaxis.get_major_formatter().set_scientific(False)  
    
    for ax in g.axes[-3:]:  
        ax.set_xlabel('Year', fontsize=10, color='black')
        ax.set_xticks(df['Year'].unique())
        ax.set_xticklabels(df['Year'].unique(), rotation=45, fontsize=10, color='black')
    
    g.set_titles(col_template="{col_name}", size=12, color='black')
    g.fig.subplots_adjust(top=0.9)
    g.fig.suptitle(f'Small Multiple Time Series for Variables', fontsize=14, color='black')
    g.add_legend(bbox_to_anchor=(1.05, 0.5), loc='center left', borderaxespad=0.)
    plt.setp(g._legend.get_texts(), fontsize='10', color='black') 

    plt.savefig(f"{output_dir}/small_plots_{chunk_index + 1}.png", bbox_inches='tight')
    plt.show()

chunk_size = 9
chunks = [variables[i:i + chunk_size] for i in range(0, len(variables), chunk_size)]

for index, chunk in enumerate(chunks):
    plot_variable_chunk(chunk, index)


In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path

output_dir = 'time_series_additional_plots'
os.makedirs(output_dir, exist_ok=True)
file_path = Path("../data/merged_canton/kantonsdaten_merged_filled.xlsx")
df = pd.read_excel(file_path, index_col=None, header=0)
df = df.rename(columns={'Jahr': 'Year', 'Kanton': 'Region'})
df = df.drop_duplicates(subset=['Year', 'Region'])
plt.style.use('default')

melted_data = df.melt(id_vars=['Year', 'Region'], var_name='Variable', value_name='Value')

variables = melted_data['Variable'].unique()

def plot_variable_chunk(chunk, chunk_index):
    chunk_data = melted_data[melted_data['Variable'].isin(chunk)]
    g = sns.FacetGrid(chunk_data, col="Variable", hue="Region", col_wrap=3, sharey=False, height=4, aspect=1.5, margin_titles=True)
    g.map(sns.lineplot, "Year", "Value")
    
    for ax in g.axes.flatten():
        ax.tick_params(labelbottom=True)
        ax.set_xticks(df['Year'].unique())
        ax.set_xticklabels(df['Year'].unique(), rotation=45)
        ax.set_xlabel('Year', fontsize=10)
        ax.set_ylabel('Value', fontsize=10)
        ax.yaxis.get_major_formatter().set_scientific(False)  

        g.set_titles(col_template="{col_name}", size=12, color='black')
    g.fig.suptitle(f'Small Multiple Time Series for Variables {chunk_index + 1}', fontsize=14, color='black', y=1.05)
    g.fig.subplots_adjust(hspace=0.4, wspace=0.2)
    g.add_legend(bbox_to_anchor=(1.05, 0.5), loc='center left', borderaxespad=0.)
    plt.savefig(f"{output_dir}/small_plots_{chunk_index + 1}.png", bbox_inches='tight')
    plt.show()

chunk_size = 15
chunks = [variables[i:i + chunk_size] for i in range(0, len(variables), chunk_size)]

for index, chunk in enumerate(chunks):
    plot_variable_chunk(chunk, index)


In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
from scipy.cluster.hierarchy import linkage, dendrogram

output_dir = 'time_series_additional_plots'
os.makedirs(output_dir, exist_ok=True)
file_path = Path("../data/merged_canton/kantonsdaten_merged_filled.xlsx")
df = pd.read_excel(file_path, index_col=None, header=0)
df = df.rename(columns={'Jahr': 'Year', 'Kanton': 'Region'})
df = df.drop_duplicates(subset=['Year', 'Region'])
plt.style.use('default')

melted_data = df.melt(id_vars=['Year', 'Region'], var_name='Variable', value_name='Value')

pivot_data = melted_data.pivot_table(index=['Year', 'Region'], columns='Variable', values='Value')
pivot_data.reset_index(inplace=True)

numeric_data = pivot_data.select_dtypes(include=[float, int])

correlation_matrix = numeric_data.corr()

linkage_matrix = linkage(correlation_matrix, method='average')
dendro = dendrogram(linkage_matrix, no_plot=True)
reordered_corr_matrix = correlation_matrix.iloc[dendro['leaves'], dendro['leaves']]

plt.figure(figsize=(20, 20))  
heatmap = sns.heatmap(reordered_corr_matrix, annot=False, cmap='coolwarm', fmt=".1f", linewidths=.5)
plt.title("Correlation Matrix of All Variables", fontsize=18, color='black')
plt.xticks(rotation=90, fontsize=12, color='black')
plt.yticks(fontsize=12, color='black')
plt.savefig(f"{output_dir}/correlation_matrix.png", bbox_inches='tight')
plt.show()
